# Проект1 
# Яндекс Музыка  

<a id='step_0'></a><h3>План работы</h3>


[1. Получение и обзор данных.](#step_1)      
[2. Предобработка данных](#step_2)     
[3. Проверка гипотез.](#step_3)    
[3.1. Действительно ли музыку в разных городах слушают по-разному?](#step_3.1)   
[3.2. Утро понедельника и вечер пятницы — разная музыка или одна и та же?](#step_3.2)  
[3.3. Москва и Питер — две разные столицы, два разных направления в музыке. Правда?](#step_3.3)    
[4. Результаты исследования](#step_4)   


<a id='step_1'></a><h3>1. Получение и обзор данных</h3>

[К плану работ](#step_0)   


Составим первое представление о данных сервиса Яндекс Музыки.

In [1]:
# импорт библиотеки pandas

import pandas as pd

Прочитаем файл *music_project.csv* и сохраним его в переменной *df*. 

In [2]:
# чтение файла с данными и сохранение в df
df = pd.read_csv('/datasets/yandex_music_project.csv')

Получение первых 10 строк таблицы.

In [3]:
# Получение первых 10 строк таблицы.
print(df.head(10))

     userID                        Track            artist   genre  \
0  FFB692EC            Kamigata To Boots  The Mass Missile    rock   
1  55204538  Delayed Because of Accident  Andreas Rönnberg    rock   
2    20EC38            Funiculì funiculà       Mario Lanza     pop   
3  A3DD03C9        Dragons in the Sunset        Fire + Ice    folk   
4  E2DC1FAE                  Soul People        Space Echo   dance   
5  842029A1                    Преданная         IMPERVTOR  rusrap   
6  4CB90AA5                         True      Roman Messer   dance   
7  F03E1C1F             Feeling This Way   Polina Griffith   dance   
8  8FA1D3BE     И вновь продолжается бой               NaN  ruspop   
9  E772D5C0                    Pessimist               NaN   dance   

             City        time        Day  
0  Saint-Petersburg  20:28:33  Wednesday  
1            Moscow  14:07:09     Friday  
2  Saint-Petersburg  20:58:07  Wednesday  
3  Saint-Petersburg  08:37:09     Monday  
4            M

Общая информация о данных таблицы *df*.

In [4]:
# Общая информация о данных таблицы df.
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65079 entries, 0 to 65078
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0     userID  65079 non-null  object
 1   Track     63848 non-null  object
 2   artist    57876 non-null  object
 3   genre     63881 non-null  object
 4     City    65079 non-null  object
 5   time      65079 non-null  object
 6   Day       65079 non-null  object
dtypes: object(7)
memory usage: 3.5+ MB


Рассмотрим полученную информацию подробнее.

Всего в таблице 7 столбцов, тип данных у каждого столбца - строковый.

Подробно разберём, какие в *df* столбцы и какую информацию они содержат:

* userID — идентификатор пользователя;
* Track — название трека;  
* artist — имя исполнителя;
* genre — название жанра;
* City — город, в котором происходило прослушивание;
* time — время, в которое пользователь слушал трек;
* Day — день недели.

Количество значений в столбцах различается. Это говорит о том, что в данных есть пропущенные значения.



**Выводы**

Каждая строка таблицы содержит информацию о композициях определённого жанра в определённом исполнении, которые пользователи слушали в одном из городов в определённое время и день недели. Две проблемы, которые нужно решать: пропуски и некачественные названия столбцов. Для проверки рабочих гипотез особенно ценны столбцы *time*, *day* и *City*. Данные из столбца *genre* позволят узнать самые популярные жанры.

<a id='step_2'></a><h3>2. Предобработка данных</h3>

[К плану работ](#step_0)   


Исключим пропуски, переименуем столбцы, а также проверим данные на наличие дубликатов.

Получаем перечень названий столбцов. Какая наблюдается проблема — кроме тех, что уже были названы ранее?

In [5]:
# перечень названий столбцов таблицы df
print(df.columns)


Index(['  userID', 'Track', 'artist', 'genre', '  City  ', 'time', 'Day'], dtype='object')


В названиях столбцов есть пробелы, которые могут затруднять доступ к данным.

Переименуем столбцы для удобства дальнейшей работы. Проверим результат.

In [6]:
# переименование столбцов
df = df.rename(columns={'  userID': 'user_id', 'Track': 'track', '  City  ': 'city', 'Day': 'day'})

In [7]:
# проверка результатов - перечень названий столбцов
print(df.columns)

Index(['user_id', 'track', 'artist', 'genre', 'city', 'time', 'day'], dtype='object')


Проверим данные на наличие пропусков вызовом набора методов для суммирования пропущенных значений.

In [8]:
# суммарное количество пропусков, выявленных методом isna() в таблице df
df.isna().sum()


user_id       0
track      1231
artist     7203
genre      1198
city          0
time          0
day           0
dtype: int64

Пустые значения свидетельствуют, что для некоторых треков доступна не вся информация. Причины могут быть разные: скажем,  не назван конкретный исполнитель народной песни. Хуже, если проблемы с записью данных. Каждый отдельный случай необходимо разобрать и выявить причину.

Заменяем пропущенные значения в столбцах с названием трека и исполнителя на строку 'unknown'. После этой операции нужно убедиться, что таблица больше не содержит пропусков.

In [9]:
# перебор названий столбцов 'track', 'artist', 'genre' в цикле и замена пропущенных значений на 'unknown'
columns_to_replace = ['track', 'artist', 'genre']
for column in columns_to_replace:
    df[column] = df[column].fillna('unknown') 

In [10]:
# проверка: вычисление суммарного количества пропусков, выявленных в таблице df
df.isna().sum()

user_id    0
track      0
artist     0
genre      0
city       0
time       0
day        0
dtype: int64

Необходимо установить наличие дубликатов.  Если найдутся, удаляем, и проверяем, все ли удалились.

In [11]:
# получение суммарного количества явных дубликатов в таблице df
df.duplicated().sum()

3826

Итого в таблице 3826 дубликатов

In [12]:
# удаление всех дубликатов из таблицы df специальным методом
df = df.drop_duplicates()

In [13]:
# проверка на отсутствие дубликатов
df.duplicated().sum()

0

Дубликаты могли появиться вследствие сбоя в записи данных. Стоит обратить внимание и разобраться с причинами появления такого «информационного мусора».

Выведем на экран список уникальных названий жанров, отсортированный в алфавитном порядке.

In [14]:
# Просмотр уникальных названий жанров
print(df['genre'].sort_values().unique())

['acid' 'acoustic' 'action' 'adult' 'africa' 'afrikaans' 'alternative'
 'alternativepunk' 'ambient' 'americana' 'animated' 'anime' 'arabesk'
 'arabic' 'arena' 'argentinetango' 'art' 'audiobook' 'author' 'avantgarde'
 'axé' 'baile' 'balkan' 'beats' 'bigroom' 'black' 'bluegrass' 'blues'
 'bollywood' 'bossa' 'brazilian' 'breakbeat' 'breaks' 'broadway'
 'cantautori' 'cantopop' 'canzone' 'caribbean' 'caucasian' 'celtic'
 'chamber' 'chanson' 'children' 'chill' 'chinese' 'choral' 'christian'
 'christmas' 'classical' 'classicmetal' 'club' 'colombian' 'comedy'
 'conjazz' 'contemporary' 'country' 'cuban' 'dance' 'dancehall' 'dancepop'
 'dark' 'death' 'deep' 'deutschrock' 'deutschspr' 'dirty' 'disco' 'dnb'
 'documentary' 'downbeat' 'downtempo' 'drum' 'dub' 'dubstep' 'eastern'
 'easy' 'electronic' 'electropop' 'emo' 'entehno' 'epicmetal' 'estrada'
 'ethnic' 'eurofolk' 'european' 'experimental' 'extrememetal' 'fado'
 'fairytail' 'film' 'fitness' 'flamenco' 'folk' 'folklore' 'folkmetal'
 'folkrock' 

In [15]:
# подсчитаем количество уникальных названий жанров
print(df['genre'].nunique())

290


Мы видим неявные дубликаты названия `hiphop`:
* *hip*,
* *hop*,
* *hip-hop*.

Чтобы очистить от них таблицу используем метод `replace()` с двумя аргументами: списком строк-дубликатов (включащий *hip*, *hop* и *hip-hop*) и строкой с правильным значением. 

In [16]:
duplicates = ['hip','hop','hip-hop']
name = 'hiphop'
# Устранение неявных дубликатов
df['genre'] = df['genre'].replace(duplicates, name)

In [17]:
# получение суммарного количества явных дубликатов в таблице df после замены неявных дубликатов
df.duplicated().sum()

0

In [18]:
# подсчитаем количество уникальных названий жанров повторно
print(df['genre'].nunique())

287


"Ушло" 3 неявных дубликата названия жанра Хипхоп

Получаем общую информацию о данных. Убеждаемся, что чистка выполнена успешно.

In [19]:
# получение общей информации о данных таблицы df
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 61253 entries, 0 to 65078
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   user_id  61253 non-null  object
 1   track    61253 non-null  object
 2   artist   61253 non-null  object
 3   genre    61253 non-null  object
 4   city     61253 non-null  object
 5   time     61253 non-null  object
 6   day      61253 non-null  object
dtypes: object(7)
memory usage: 3.7+ MB


**Выводы**

Предобработка обнаружила три проблемы в данных:

- нарушения в стиле оформления заголовков,
- пропущенные значения,
- дубликаты — явные и неявные.

мы исправили заголовки, чтобы упростить работу с таблицей. Без дубликатов исследование станет более точным.  
Пропущенные значения мы заменили на `'unknown'`. Ещё предстоит увидеть, не повредят ли исследованию пропуски в колонке `genre`.   
Поскольку сведения о жанрах важно сохранить для анализа, не просто удалили все пропущенные значения, а заполнили пропущенные имена исполнителей и названия треков и жанров. Имена столбцов теперь корректны и удобны для дальнейшей работы.    
Теперь можно перейти к проверке гипотез.  

<a id='step_3'></a><h3>3. Проверка гипотез</h3>

[К плану работ](#step_0)   

<a id='step_3.1'></a><h3>3.1 Действительно ли музыку в разных городах слушают по-разному?</h3>

[К плану работ](#step_0) 

Была выдвинута гипотеза, что в Москве и Санкт-Петербурге пользователи слушают музыку по-разному. Проверяем это предположение по данным о трёх днях недели — понедельнике, среде и пятнице.

Для каждого города устанавливаем количество прослушанных  в эти дни композиций с известным жанром, и сравниваем результаты.

Группируем данные по городу и вызовом метода *count()* подсчитываем композиции, для которых известен жанр.

In [20]:
# группировка данных таблицы df по столбцу 'city' и подсчёт количества значений столбца 'genre'
city_grouping = df.groupby('city')['genre'].count()
print(city_grouping)
# Подсчёт прослушиваний в каждом городе

city
Moscow              42741
Saint-Petersburg    18512
Name: genre, dtype: int64


In [21]:
# группировка данных таблицы df по столбцу 'city' и подсчёт количества уникальных пользователей значений столбца 'user_id'
city_grouping2 = df.groupby('city')['user_id'].nunique()
print(city_grouping2)# подсчет уникальных слушателей в Москве и Санкт-Петербурге

city
Moscow              29358
Saint-Petersburg    12423
Name: user_id, dtype: int64


В Москве прослушиваний больше, чем в Питере, но это не значит, что Москва более активна. У Яндекс.Музыки в целом больше пользователей в Москве, поэтому величины сопоставимы.

Сгруппируем данные по дню недели и подсчитаем прослушанные в понедельник, среду и пятницу композиции, для которых известен жанр.

In [22]:
# группировка данных по столбцу 'day' и подсчёт количества значений столбца 'city'
day_grouping = df.groupby('day')['city'].count()
print(day_grouping)# Подсчёт прослушиваний в каждый из трёх дней

day
Friday       21840
Monday       21354
Wednesday    18059
Name: city, dtype: int64


В среднем пользователи из двух городов менее активны по средам. Но картина может измениться, если рассмотреть каждый город в отдельности.

Создаём функцию number_tracks(), которая принимает как параметры таблицу, день недели и название города, а возвращает количество прослушанных композиций, для которых известен жанр. Проверяем количество прослушанных композиций для каждого города и понедельника, затем среды и пятницы.

In [23]:
# создание функции number_tracks()
# объявляется функция с тремя параметрами: df, day, city
def number_tracks(day, city):
    # в переменной track_list_day сохраняются те строки таблицы df, для которых 
    # значение в столбце 'day' равно параметру day
    track_list_day = df[df['day'] == day]
    # и одновременно значение в столбце 'city' равно параметру city
    track_list = track_list_day[track_list_day['city'] == city]
    # в переменной track_list_count сохраняется число значений столбца 'user_id',
    # рассчитанное методом count() для таблицы track_list
    track_list_count = track_list['user_id'].count()
    # функция возвращает значение track_list_count    
    return track_list_count


# проверим как работает функция, подставив Понедельник и Москва
number_tracks('Monday', 'Moscow') 

15740

In [24]:
# список композиций для Москвы в понедельник
mm = number_tracks('Monday', 'Moscow')
mm

15740

In [25]:
# список композиций для Санкт-Петербурга в понедельник
sm = number_tracks('Monday', 'Saint-Petersburg')
sm

5614

In [26]:
# список композиций для Москвы в среду
mw = number_tracks('Wednesday', 'Moscow')
mw

11056

In [27]:
# список композиций для Санкт-Петербурга в среду
sw = number_tracks('Wednesday', 'Saint-Petersburg')
sw

7003

In [28]:
# список композиций для Москвы в пятницу
mf = number_tracks('Friday', 'Moscow')
mf

15945

In [29]:
# список композиций для Санкт-Петербурга в пятницу
sf = number_tracks('Friday', 'Saint-Petersburg')
sf

5895

Сведём полученную информацию в одну таблицу, где ['city', 'monday', 'wednesday', 'friday'] названия столбцов.

In [30]:
# таблица с полученными данными
info = pd.DataFrame(data=[['Москва', mm, mw, mf], ['Санкт-Петербург', sm, sw, sf]], 
                    columns=['city', 'monday', 'wednesday', 'friday'])
info
# Таблица с результатами

,city,monday,wednesday,friday
0,Москва,15740,11056,15945
1,Санкт-Петербург,5614,7003,5895


**Вывод**

Результаты показывают, что относительно среды музыку в Петербурге и Москве слушают «зеркально»: в Москве пики приходятся на понедельник и пятницу, а в среду время прослушивания снижается. Тогда как в Санкт-Петербурге среда — день самого большого интереса к музыке, а в понедельник и пятницу он меньше, причём почти одинаково меньше.

<a id='step_3.2'></a><h3>3.2 Утро понедельника и вечер пятницы — разная музыка или одна и та же?</h3>

[К плану работ](#step_0) 

Ищем ответ на вопрос, какие жанры преобладают в разных городах в понедельник утром и в пятницу вечером. Есть предположение, что в понедельник утром пользователи слушают больше бодрящей музыки (например, жанра поп), а вечером пятницы — больше танцевальных (например, электронику).

Получим таблицы данных по Москве *moscow_general* и по Санкт-Петербургу *spb_general*.

In [31]:
# получение таблицы moscow_general из тех строк таблицы df, 
# для которых значение в столбце 'city' равно 'Moscow'
moscow_general = df[df['city'] == 'Moscow']
print(moscow_general)


        user_id                          track                   artist  \
1      55204538    Delayed Because of Accident         Andreas Rönnberg   
4      E2DC1FAE                    Soul People               Space Echo   
6      4CB90AA5                           True             Roman Messer   
7      F03E1C1F               Feeling This Way          Polina Griffith   
8      8FA1D3BE       И вновь продолжается бой                  unknown   
...         ...                            ...                      ...   
65073  83A474E7  I Worship Only What You Bleed  The Black Dahlia Murder   
65074  729CBB09                        My Name                   McLean   
65076  C5E3A0D5                      Jalopiina                  unknown   
65077  321D0506                  Freight Train            Chas McDevitt   
65078  3A64EF84      Tell Me Sweet Little Lies             Monica Lopez   

              genre    city      time        day  
1              rock  Moscow  14:07:09     Friday

In [32]:
# получение таблицы spb_general
# для которых значение в столбце 'city' равно 'Saint-Petersburg'

spb_general = df[df['city'] == 'Saint-Petersburg']
print(spb_general)

        user_id                              track              artist  \
0      FFB692EC                  Kamigata To Boots    The Mass Missile   
2        20EC38                  Funiculì funiculà         Mario Lanza   
3      A3DD03C9              Dragons in the Sunset          Fire + Ice   
5      842029A1                          Преданная           IMPERVTOR   
9      E772D5C0                          Pessimist             unknown   
...         ...                                ...                 ...   
65063  D94F810B        Theme from the Walking Dead  Proyecto Halloween   
65064  BC8EC5CF       Red Lips: Gta (Rover Rework)               Rover   
65065  29E04611                       Bre Petrunko       Perunika Trio   
65066  1B91C621             (Hello) Cloud Mountain     sleepmakeswaves   
65075  D08D4A55  Maybe One Day (feat. Black Spade)         Blu & Exile   

            genre              city      time        day  
0            rock  Saint-Petersburg  20:28:33  Wedne

Создаём функцию *genre_weekday()*, которая возвращает список жанров по запрошенному дню недели и времени суток с такого-то часа по такой-то.

In [33]:
# объявление функции genre_weekday() с параметрами df, day, time1, time2
def genre_weekday(df, day, time1, time2):
    # в переменной genre_df сохраняются те строки df, для которых одновременно:
    # 1) значение в столбце 'day' равно параметру day,
    genre_df = df[df['day'] == day]
    # 2) значение в столбце 'time' больше time1 и
    genre_df =  genre_df[ genre_df['time'] > time1]
    # 3) меньше time2.
    genre_df =  genre_df[ genre_df['time'] < time2]
    # в переменной genre_df_grouped сохраняется  Series, полученная подсчётом числа значений столбца 'genre' 
    genre_df_grouped = genre_df.groupby('genre')['genre'].count()
    # отсортированной по убыванию таблицы genre_df_grouped
    genre_df_sorted = genre_df_grouped.sort_values(ascending=False)
    # функция возвращает значение genre_df_sorted, первые 10 значений 
    return genre_df_sorted[:10]


Cравниваем полученные результаты по таблице для Москвы и Санкт-Петербурга в понедельник утром (с 7 до 11) и в пятницу вечером (с 17 до 23).

In [34]:
# вызов функции для утра понедельника в Москве (вместо df таблица moscow_general)
genre_weekday(moscow_general, 'Monday', '07:00', '11:00')

genre
pop            781
dance          549
electronic     480
rock           474
hiphop         286
ruspop         186
world          181
rusrap         175
alternative    164
unknown        161
Name: genre, dtype: int64

In [35]:
# вызов функции для утра понедельника в Петербурге (вместо df таблица spb_general)
genre_weekday(spb_general, 'Monday', '07:00', '11:00')

genre
pop            218
dance          182
rock           162
electronic     147
hiphop          80
ruspop          64
alternative     58
rusrap          55
jazz            44
classical       40
Name: genre, dtype: int64

In [36]:
# вызов функции для вечера пятницы в Москве
genre_weekday(moscow_general, 'Friday', '17:00', '23:00')

genre
pop            713
rock           517
dance          495
electronic     482
hiphop         273
world          208
ruspop         170
alternative    163
classical      163
rusrap         142
Name: genre, dtype: int64

In [37]:
# вызов функции для вечера пятницы в Питере
genre_weekday(spb_general, 'Friday', '17:00', '23:00')

genre
pop            256
electronic     216
rock           216
dance          210
hiphop          97
alternative     63
jazz            61
classical       60
rusrap          59
world           54
Name: genre, dtype: int64

Популярные жанры в понедельник утром в Питере и Москве оказались похожи: везде, как и предполагалось, популярен поп. Несмотря на это, концовка топ-10 для двух городов различается: в Питере в топ-10 входит классика и джаз, а в Москве жанр *alternative*. Кроме того в Москве пропущенных значений оказалось так много, что значение 'unknown' заняло десятое место среди самых популярных жанров. Значит, пропущенные значения занимают существенную долю в данных и угрожают достоверности исследования.

Вечер пятницы не меняет эту картину. Некоторые жанры поднимаются немного выше, другие спускаются, но в целом топ-10 остаётся тем же самым.

Таким образом, вторая гипотеза подтвердилась лишь частично:

Пользователи слушают похожую музыку в начале недели и в конце.
Разница между Москвой и Петербургом не слишком выражена. В Москве чаще слушают русскую популярную музыку, в Петербурге — джаз.
Однако пропуски в данных ставят под сомнение этот результат. В Москве их так много, что рейтинг топ-10 мог бы выглядеть иначе, если бы не утерянные данные о жанрах.

**Вывод**

Жанр поп безусловный лидер, а топ-5 в целом не различается в обеих столицах. При этом видно, что концовка списка более «живая»: для каждого города выделяются более характерные жанры, которые действительно меняют свои позиции в зависимости от дня недели и времени.

<a id='step_3.3'></a><h3>3.3 Москва и Питер — две разные столицы, два разных направления в музыке. Правда?</h3>

[К плану работ](#step_0)  

Гипотеза: Питер богат своей рэп-культурой, поэтому это направление там слушают чаще, а Москва — город контрастов, но основная масса пользователей слушает попсу.


Сгруппируем таблицу *moscow_general* по жанру, сосчитаем численность композиций каждого жанра методом *count()*, отсортируем в порядке убывания и сохраним результат в таблице *moscow_genres*.

Просмотрим первые 10 строк этой новой таблицы.

In [38]:
# одной строкой: группировка таблицы moscow_general по столбцу 'genre', 
# подсчёт числа значений 'genre' в этой группировке методом count(), 
# сортировка Series в порядке убывания и сохранение в moscow_genres
moscow_genres = moscow_general.groupby('genre')['genre'].count().sort_values(ascending=False)

In [39]:
# просмотр первых 10 строк moscow_genres
print(moscow_genres.head(10))

genre
pop            5892
dance          4435
rock           3965
electronic     3786
hiphop         2096
classical      1616
world          1432
alternative    1379
ruspop         1372
rusrap         1161
Name: genre, dtype: int64


Сгруппируем таблицу *spb_general* по жанру, сосчитаем численность композиций каждого жанра методом *count()*, отсортируем в порядке убывания и сохраним результат в таблице *spb_genres*.

Просматриваем первые 10 строк этой таблицы. Теперь можно сравнивать два города.

In [40]:
# группировка таблицы spb_general, расчёт, сохранение в spb_genres
spb_genres = spb_general.groupby('genre')['genre'].count().sort_values(ascending=False)

In [41]:
# просмотр первых 10 строк spb_genres
print(spb_genres.head(10))

genre
pop            2431
dance          1932
rock           1879
electronic     1736
hiphop          960
alternative     649
classical       646
rusrap          564
ruspop          538
world           515
Name: genre, dtype: int64


**Вывод**

Гипотеза частично подтвердилась:

Поп-музыка — самый популярный жанр в Москве, как и предполагала гипотеза. Более того, в топ-10 жанров встречается близкий жанр — русская популярная музыка.
Вопреки ожиданиям, рэп одинаково популярен в Москве и Петербурге.

<a id='step_4'></a><h3>4. Результаты исследования</h3>

[К плану работ](#step_0)   

Рабочие гипотезы:

* музыку в двух городах — Москве и Санкт-Петербурге — слушают в разном режиме;

* списки десяти самых популярных жанров утром в понедельник и вечером в пятницу имеют характерные отличия;

* население двух городов предпочитает разные музыкальные жанры.   


1. День недели по-разному влияет на активность пользователей в Москве и Петербурге. 
2. Музыкальные предпочтения не сильно меняются в течение недели — будь то Москва или Петербург. Небольшие различия заметны в начале недели, по понедельникам:
* в Москве слушают музыку жанра “world”,
* в Петербурге — джаз и классику.
3. Во вкусах пользователей Москвы и Петербурга больше общего чем различий. Вопреки ожиданиям, предпочтения жанров в Петербурге напоминают московские.


**Общие результаты**

Москва и Петербург сходятся во вкусах: везде преобладает популярная музыка. При этом зависимости предпочтений от дня недели в каждом отдельном городе нет — люди постоянно слушают то, что им нравится. Но между городами в разрезе дней недели наблюдается зеркальность относительно среды: Москва больше слушает в понедельник и пятницу, а Петербург наоборот - больше в среду, но меньше в понедельник и пятницу.

В результате первая гипотеза полностью подтвердилась, вторая гипотеза вторая гипотеза подтвердилась лишь отчасти и третья не подтверждена.


